In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/Colab Notebooks/chet-Xray"

/content/drive/MyDrive/Colab Notebooks/chet-Xray


In [4]:
import flask
print(flask.__version__)

1.1.4


In [5]:
# Check tensorflow version and GPU availability
import tensorflow as tf
print('tf version:', tf.__version__)
print('gpu:', tf.config.list_physical_devices('GPU'))

tf version: 2.6.0
gpu: []


In [6]:
IMG_DIMEN = 224
N_COLORS = 3   

In [7]:
import cv2
import numpy as np

def read_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, dsize=(IMG_DIMEN, IMG_DIMEN))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype('float32')/255.0
    return img

In [8]:
import matplotlib.pyplot as plt
%matplotlib inline

def imshow(img):
    plt.figure(figsize=(6, 6))
    if N_COLORS == 3:
        plt.imshow(img)
    elif N_COLORS == 1:
         plt.imshow(img, cmap='gray')
    plt.show()

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Dense

pretrained_model = MobileNet(
    include_top=False,
    input_shape=(IMG_DIMEN, IMG_DIMEN, N_COLORS), 
    pooling='avg', 
)

pretrained_model.trainable = False

model = Sequential()
model.add(pretrained_model)
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

17235968/17225924 [==============================] - 0s 0us/step


In [10]:
model.load_weights('saved-model-mobilenet/best-model')

In [11]:
def classify_image(img_path):
    img = read_image(img_path)
    return float(model.predict(np.array([img])).squeeze())

In [ ]:
from flask import Flask
from flask import request

app = Flask(__name__)

@app.route('/classify', methods = ['POST'])
def classify_handler():
    img_path = request.form['path']
    y_pred = classify_image(img_path)
    class_name = 'PNEUMONIA' if y_pred >= 0.5 else 'NORMAL'
    prob_malignant = y_pred
    prob_benign = 1 - y_pred
    print('img_path:', img_path)
    print('y_pred:', y_pred)
    print('class_name:', class_name)
    return {
        'prob_NORMAL': prob_NORMAL, 
        'prob_PNEUMONIA': prob_PNEUMONIA, 
        'class_name': class_name
    }

app.run(port=9000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:9000/ (Press CTRL+C to quit)
